In [1]:
# Imports principales para el forecasting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays


In [2]:
# Cargar archivo de ventas para inferencia
inferencia_path = "../data/raw/inferencia/ventas_2025_inferencia.csv"
inferencia_df = pd.read_csv(inferencia_path)

# Mostrar las primeras filas para verificar la carga
display(inferencia_df.head())


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [3]:
# --- Transformación completa de inferencia_df para inferencia ---

# 1. Convertir fecha a datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'], errors='coerce')

# 2. Crear variables temporales
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['semana'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['semana_año'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter

# 3. Día de la semana y si es fin de semana
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin(['Saturday', 'Sunday'])

# 4. Festivos España
es_holidays = holidays.country_holidays('ES', years=inferencia_df['año'].unique())
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays)

# 5. Black Friday y Cyber Monday
def es_black_friday(fecha):
    if pd.isnull(fecha): return False
    if fecha.month != 11: return False
    # Último viernes de noviembre
    nov = pd.date_range(start=f'{fecha.year}-11-01', end=f'{fecha.year}-11-30', freq='D')
    fridays = nov[nov.weekday == 4]
    return fecha == fridays[-1]

def es_cyber_monday(fecha):
    if pd.isnull(fecha): return False
    if fecha.month != 11: return False
    # Cyber Monday: lunes siguiente al Black Friday
    nov = pd.date_range(start=f'{fecha.year}-11-01', end=f'{fecha.year}-11-30', freq='D')
    fridays = nov[nov.weekday == 4]
    if len(fridays) == 0: return False
    cyber_monday = fridays[-1] + pd.Timedelta(days=3)
    return fecha == cyber_monday

inferencia_df['es_Black_Friday'] = inferencia_df['fecha'].apply(es_black_friday)
inferencia_df['es_Cyber_Monday'] = inferencia_df['fecha'].apply(es_cyber_monday)

# 6. Primer y último día del mes
inferencia_df['es_primer_dia_mes'] = inferencia_df['fecha'].dt.is_month_start
inferencia_df['es_ultimo_dia_mes'] = inferencia_df['fecha'].dt.is_month_end

# 7. Día laboral (no festivo y no fin de semana)
inferencia_df['es_dia_laboral'] = (~inferencia_df['es_festivo']) & (~inferencia_df['es_fin_semana'])

# 8. Primera mitad del mes
inferencia_df['es_primera_mitad'] = inferencia_df['dia_mes'] <= 15

# 9. Navidad y Reyes
inferencia_df['es_navidad'] = inferencia_df['fecha'].dt.month.eq(12) & inferencia_df['fecha'].dt.day.eq(25)
inferencia_df['es_reyes'] = inferencia_df['fecha'].dt.month.eq(1) & inferencia_df['fecha'].dt.day.eq(6)

# 10. Laborable, inicio y fin de mes
inferencia_df['es_laborable'] = inferencia_df['es_dia_laboral']
inferencia_df['es_inicio_mes'] = inferencia_df['es_primer_dia_mes']
inferencia_df['es_fin_mes'] = inferencia_df['es_ultimo_dia_mes']

# 11. Lags y medias móviles (rellenar con NaN, ya que no hay historial en inferencia)
for lag in range(1,8):
    inferencia_df[f'unidades_vendidas_lag{lag}'] = np.nan
inferencia_df['unidades_vendidas_ma7'] = np.nan

# 12. Descuento y precio competencia
inferencia_df['descuento_porcentaje'] = (inferencia_df['precio_base'] - inferencia_df['precio_venta']) / inferencia_df['precio_base']
inferencia_df['precio_competencia'] = inferencia_df[['Amazon','Decathlon','Deporvillage']].mean(axis=1)
inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# 13. One-hot encoding de nombre, categoría y subcategoría
nombres = [
    'Adidas Own The Run Jacket', 'Adidas Ultraboost 23', 'Asics Gel Nimbus 25', 'Bowflex SelectTech 552',
    'Columbia Silver Ridge', 'Decathlon Bandas Elásticas Set', 'Domyos BM900', 'Domyos Kit Mancuernas 20kg',
    'Gaiam Premium Yoga Block', 'Liforme Yoga Pad', 'Lotuscrafts Yoga Bolster', 'Manduka PRO Yoga Mat',
    'Merrell Moab 2 GTX', 'New Balance Fresh Foam X 1080v12', 'Nike Air Zoom Pegasus 40', 'Nike Dri-FIT Miler',
    'Puma Velocity Nitro 2', 'Quechua MH500', 'Reebok Floatride Energy 5', 'Reebok Professional Deck',
    'Salomon Speedcross 5 GTX', 'Sveltus Kettlebell 12kg', 'The North Face Borealis', 'Trek Marlin 7'
]
for nombre in nombres:
    inferencia_df[f'nombre_h_{nombre}'] = (inferencia_df['nombre'] == nombre).astype(int)

categorias = ['Fitness', 'Outdoor', 'Running', 'Wellness']
for cat in categorias:
    inferencia_df[f'categoria_h_{cat}'] = (inferencia_df['categoria'] == cat).astype(int)

subcategorias = [
    'Banco Gimnasio', 'Bandas Elásticas', 'Bicicleta Montaña', 'Bloque Yoga', 'Cojín Yoga', 'Esterilla Fitness',
    'Esterilla Yoga', 'Mancuernas Ajustables', 'Mochila Trekking', 'Pesa Rusa', 'Pesas Casa', 'Rodillera Yoga',
    'Ropa Montaña', 'Ropa Running', 'Zapatillas Running', 'Zapatillas Trail'
]
for subcat in subcategorias:
    inferencia_df[f'subcategoria_h_{subcat}'] = (inferencia_df['subcategoria'] == subcat).astype(int)

# 14. Eliminar registros de octubre y dejar solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

# 15. Guardar el dataframe transformado
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)

# Mostrar las primeras filas para verificar
display(inferencia_df.head())

C:\Users\jhost\AppData\Local\Temp\ipykernel_22844\1506090635.py:21: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays)


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Esterilla Yoga,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail
168,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,...,0,0,0,0,0,0,0,0,1,0
169,2025-11-01,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,NaN,135.00,NaN,...,0,0,0,0,0,0,0,0,1,0
170,2025-11-01,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,NaN,86.39,NaN,...,0,0,0,0,0,0,0,0,1,0
171,2025-11-01,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,NaN,74.09,NaN,...,0,0,0,0,0,0,0,0,1,0
172,2025-11-01,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,NaN,34.76,NaN,...,0,0,0,0,0,0,0,1,0,0


In [10]:
inferencia_df.shape

(720, 89)

In [11]:
# Guardar el DataFrame transformado en data/processed
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)
print("DataFrame transformado guardado en data/processed/inferencia_df_transformado.csv")

# Mostrar resumen final
print("\nResumen final:")
print(f"Total de registros: {len(inferencia_df)}")
print(f"Total de columnas: {len(inferencia_df.columns)}")
print(f"Productos únicos: {inferencia_df['producto_id'].nunique()}")
print(f"Rango de fechas: {inferencia_df['fecha'].min()} a {inferencia_df['fecha'].max()}")


DataFrame transformado guardado en data/processed/inferencia_df_transformado.csv

Resumen final:
Total de registros: 720
Total de columnas: 89
Productos únicos: 24
Rango de fechas: 2025-11-01 00:00:00 a 2025-11-30 00:00:00
